In [ ]:
!pip install opencv-python tf-models-official "tensorflow-text==2.9.*"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/ml/* ./

In [ ]:
# 数据量中间计算量过大，将训练数据转换成TFRecord。省去中间重复计算（tokenid,typeid,mkid）的时间。
# !git clone https://github.com/tensorflow/models.git
# !python3 models/official/nlp/data/create_finetuning_data.py \
#  --squad_data_file=/content/squad/train-v2.0.json \
#  --vocab_file=gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12/vocab.txt \
#  --train_data_output_path=/content/squad/train_v2.0.tf_record \
#  --meta_data_file_path=/content/squad/squad_v2.0_meta_data \
#  --fine_tuning_task_type=squad \
#  --max_seq_length=384 \
#  --version_2_with_negative=True

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow_models import nlp
from keras import metrics, layers, optimizers, losses, Model
from keras.utils import plot_model
import json

In [ ]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

In [ ]:
database_dir = "/content"
bert_dir = gs_folder_bert
squad_version = "v2.0"
squad_dir = database_dir+"/squad"
max_seq_length = 384
train_batch_size = 12
train_epochs=5

In [ ]:
raw_dataset = tf.data.TFRecordDataset([os.path.join(
    squad_dir, f"train_{squad_version}.tf_record")])

feature_description = {
    'unique_ids': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'input_ids': tf.io.FixedLenFeature(shape=(max_seq_length,), dtype=tf.int64, default_value=[0] * max_seq_length),
    'input_mask': tf.io.FixedLenFeature(shape=(max_seq_length,), dtype=tf.int64, default_value=[0] * max_seq_length),
    'segment_ids': tf.io.FixedLenFeature(shape=(max_seq_length,), dtype=tf.int64, default_value=[0] * max_seq_length),
    'paragraph_mask': tf.io.FixedLenFeature(shape=(max_seq_length,), dtype=tf.int64, default_value=[0] * max_seq_length),
    'class_index': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'start_positions': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'end_positions': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'is_impossible': tf.io.FixedLenFeature([], tf.int64, default_value=0)
}

def decode_fn(record_bytes):
    example = tf.io.parse_single_example(
        record_bytes,
        feature_description
    )
    # example["input_word_ids"] = example["input_ids"]
    # example["input_type_ids"] = example["segment_ids"]

    return {
        'input_word_ids': example["input_ids"],
        'input_type_ids': example["segment_ids"],
        'input_mask': example["input_mask"],
        'is_impossible': example["is_impossible"]
    }, {
        "start_positions": example["start_positions"],
        "end_positions": example["end_positions"],
    }


# for raw_record in raw_dataset.take(10).map(decode_fn):
#     print(raw_record)

bert_config_file = os.path.join(bert_dir, 'bert_config.json')
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

dataset = raw_dataset.shuffle(buffer_size=train_batch_size).map(decode_fn)


In [ ]:
# 总条目131944,可用条目87212
train_num = 70000

dataset = dataset.filter(lambda x, y: x['is_impossible'] == 0)

train_ds = dataset.take(train_num).batch(batch_size=train_batch_size)

test_ds = dataset.skip(train_num).batch(batch_size=train_batch_size)

In [ ]:
encoder_config = nlp.encoders.EncoderConfig({
    'type': 'bert',
    'bert': config_dict
})

bert_encoder = nlp.encoders.build_encoder(encoder_config)

bert_span = nlp.models.BertSpanLabeler(network=bert_encoder)

In [ ]:
plot_model(bert_span, show_shapes=True)

In [ ]:
checkpoint = tf.train.Checkpoint(encoder=bert_encoder)

checkpoint.read(
    os.path.join(bert_dir, 'bert_model.ckpt')
).assert_consumed()

In [ ]:
bert_span.compile(optimizer='adam',
                  loss={
                      'start_positions': 'sparse_categorical_crossentropy',
                      'end_positions': 'sparse_categorical_crossentropy'
                  },
                  metrics=['accuracy'])

In [ ]:
bert_span.evaluate(test_ds)

In [ ]:
bert_span.fit(train_ds,
              validation_data=(test_ds),
              batch_size=train_batch_size,
              epochs=train_epochs)